In [ ]:
import asyncio
from dotenv import load_dotenv
import os

# 确保环境变量已加载 (特别是 GEMINI_API_KEY)
# 假设 .env 文件位于 notebook 所在的 backend 目录的上一级 (即项目根目录)
# 如果 .env 在 backend 目录中，路径应为 '.env' 或 '../.env' (如果notebook在src/agent下)
# 根据您的 .env 文件实际位置调整 dotenv_path
# 您之前提到 .env 在 backend 目录中，所以应该是 load_dotenv() 或 load_dotenv(dotenv_path='.env')
# 如果 .env 在项目根目录，则是 load_dotenv(dotenv_path='../.env')
# 从您打开的文件看，.env 在 backend 目录，所以 load_dotenv() 应该可以。
if load_dotenv(): # 或者 load_dotenv(dotenv_path='.env')
    print("Loaded .env file")
else:
    print("Could not load .env file, ensure GEMINI_API_KEY is set in your environment")


In [ ]:
from langgraph.graph import StateGraph
#help(StateGraph)
from langchain_core.messages import AIMessage
help(AIMessage)

In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import os

DEFAULT_SEARCH_ENGINE_TIMEOUT = 100
REFERENCE_COUNT = 5
GOOGLE_SEARCH_ENDPOINT = "https://customsearch.googleapis.com/customsearch/v1"

def search_with_google(query: str, subscription_key: str, cx: str):
    """
    Search with google and return the contexts.
    """
    params = {
        "key": subscription_key,
        "cx": cx,
        "q": query,
        "num": REFERENCE_COUNT,
    }
    response = requests.get(
        GOOGLE_SEARCH_ENDPOINT, params=params, timeout=DEFAULT_SEARCH_ENGINE_TIMEOUT
    )
    if not response.ok:
        print(f"{response.status_code} {response.text}")
        raise HTTPException(response.status_code, "Search engine error.")
    json_content = response.json()
    try:
        contexts = json_content["items"][:REFERENCE_COUNT]
    except KeyError:
        print(f"Error encountered: {json_content}")
        return []
    return contexts


In [ ]:
search_api_key = os.environ["GOOGLE_SEARCH_API_KEY"]
# with DDGS() as ddgs:
#     print(query, "xxx")
#     results = list(ddgs.text(query, max_results=5))

results = search_with_google("中国的首都", search_api_key, 
os.environ["GOOGLE_SEARCH_CX"])

In [ ]:
print(results)

In [ ]:

# 打印一下关键环境变量，确认是否加载成功
print(f"GEMINI_API_KEY is set: {bool(os.getenv('GEMINI_API_KEY'))}")
print(f"LANGSMITH_API_KEY is set: {bool(os.getenv('LANGSMITH_API_KEY'))}") 
print(f"GEMINI_API_KEY is : {os.getenv('GEMINI_API_KEY')}") # 如果 LangSmith 也需要配置


In [ ]:
from langchain_core.messages import HumanMessage
import os, dotenv, pathlib

# 加载 .env（若在 backend 目录，可写 dotenv.load_dotenv("backend/.env")）
dotenv.load_dotenv()

from langchain_openai import AzureChatOpenAI

# 创建 LLM 实例
llm = AzureChatOpenAI(
    deployment_name='gpt-4.1',
    model_name='gpt-4.1',
    temperature=0.2,
    streaming=True,
)
from langchain_core.messages import HumanMessage
response = llm.invoke([HumanMessage(content="hello")])
print(response)

In [ ]:
from google.genai import Client

genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))
help(genai_client)

In [ ]:
import logging
import sys
from IPython.display import display, HTML
import asyncio
import nest_asyncio
nest_asyncio.apply()

# Configure logging
logging.basicConfig(
    stream=sys.stdout, level=logging.DEBUG
)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from src.agent.graph import graph
from src.agent.configuration import Configuration
import importlib
import inspect

# 重新载入模块以确保获取最新代码
importlib.reload(importlib.import_module('src.agent.graph'))
from src.agent.graph import graph

# 打印 Configuration 类的所有字段及其默认值
print("Fields in Configuration class:")
for field_name, field_obj in Configuration.model_fields.items():
    print(f"- {field_name}: (default: {field_obj.default})")

print("\n" + "="*50 + "\n")

# 尝试创建一个 Configuration 实例并打印其内容
# 这也会验证 from_runnable_config 是否能正常工作（如果环境变量未设置，则使用默认值）
try:
    config_instance = Configuration.from_runnable_config()
    print("Created Configuration instance with current settings:")
    print(config_instance.model_dump_json(indent=2))
except Exception as e:
    print(f"Error creating Configuration instance: {e}")

print("\n" + "="*50 + "\n")

# 检查模块的加载路径
print("Path from where 'src.agent.configuration' was loaded:")
print(inspect.getfile(Configuration))


In [ ]:
state = graph.invoke({"messages": [{"role": "user", "content": "中国的首都"}], "max_research_loops": 3, "initial_search_query_count": 1})


In [ ]:
import os
from google.genai import Client

# Initialize the Gemini client
genai_client = Client(api_key=os.getenv("GEMINI_API_KEY"))

# Test a simple query
print(os.getenv("GEMINI_API_KEY"))
response = genai_client.models.generate_content(
    model="gemini-2.0-flash",
    contents="hello",
)
print(response.text)

In [ ]:

import asyncio
from dotenv import load_dotenv
import os
from pprint import pprint # For pretty printing dictionaries

# 确保环境变量已加载 (特别是 GEMINI_API_KEY)
if load_dotenv(): # 或者 load_dotenv(dotenv_path='.env')
    print("Loaded .env file")
else:
    print("Could not load .env file, ensure GEMINI_API_KEY is set in your environment")

print(f"GEMINI_API_KEY is set: {bool(os.getenv('GEMINI_API_KEY'))}")
print(f"LANGSMITH_API_KEY is set: {bool(os.getenv('LANGSMITH_API_KEY'))}")

try:
    from src.agent.graph import graph
    print("Successfully imported 'graph' from src.agent.graph")
except ImportError as e:
    print(f"Error importing graph: {e}")
    graph = None

async def main():
    if not graph:
        print("Graph not loaded, skipping invocation.")
        return

    initial_state = {
        "messages": [
            {"role": "user", "content": "Who won the UEFA Euro 2024 football tournament?"}
        ],
        "max_research_loops": 2,
        "initial_search_query_count": 2
    }

    run_config = {
        "configurable": {
            # "query_generator_model": "gemini-1.5-flash-latest",
            # "reasoning_model": "gemini-1.5-pro-latest",
        }
    }
    
    print("\nStreaming agent execution log with initial state:")
    print(initial_state)
    if run_config.get("configurable"):
        print("\nAnd config:")
        print(run_config)
    print("\n--- Agent Log Start ---")

    try:
        # 使用 astream_log() 来获取详细的执行日志
        async for log_patch in graph.astream_log(
            initial_state, 
            config=run_config, 
            include_names=None, 
            include_types=None, 
        ):
            # RunLogPatch 对象包含一个 'ops' 列表，每个 op 是一个操作描述
            for op in log_patch.ops:
                print(f"\n[Log Operation: {op['op']}] Path: {op['path']}")
                if op['op'] == "add" or op['op'] == "replace":
                    # 'value' 字段包含了添加或替换的数据
                    print("  Value:")
                    pprint(op['value'], indent=2, width=120)
                elif op['op'] == "remove":
                    print(f"  (Removed path: {op['path']})")
                # 可以根据 op['op'] 的类型 ('add', 'remove', 'replace') 和 op['path'] 来决定如何展示信息
                # 例如，关心特定路径的更新：
                # if op['path'] == "/logs/OverallState/final_output" and op['op'] == 'add':
                #     print(">>> Final Output Updated <<<")
                #     pprint(op['value'])
                # if "streamed_output" in op['path'] and op['op'] == 'add': # LLM token stream
                #     if isinstance(op['value'], list) and op['value'] and hasattr(op['value'][0], 'content'):
                #         print(f"  LLM Stream: {op['value'][0].content}", end="") # 假设是 AIMessageChunk
                #     else:
                #         pprint(op['value'], indent=2)


            print("-" * 40)
        
        print("\n--- Agent Log End ---")
        
    except Exception as e:
        print(f"\nAn error occurred during agent streaming: {e}")
        import traceback
        traceback.print_exc()

# **在 Notebook 单元格中，请直接运行:**
await main()

In [ ]:
state

In [ ]:
from IPython.display import Markdown

Markdown(state["messages"][-1].content)

In [ ]:
state = graph.invoke({"messages": state["messages"] + [{"role": "user", "content": "How has the most titles? List the top 5"}]})

In [ ]:
Markdown(state["messages"][-1].content)

In [ ]:
"""
测试脚本，用于单独测试 generate_query 函数
"""

import os
from dotenv import load_dotenv
from src.agent.graph import generate_query
from src.agent.configuration import Configuration
from langchain_core.runnables import RunnableConfig
from langchain_core.messages import HumanMessage

# 加载环境变量
load_dotenv()

# 检查API密钥是否设置
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    print("GEMINI_API_KEY 未设置")
    exit(1)
else:
    print("GEMINI_API_KEY 已设置")

# 创建测试状态
test_state = {
    "messages": [
        HumanMessage(content="谁赢得了2024年欧洲杯足球赛?")
    ]
}

# 创建配置
test_config = RunnableConfig(
    configurable={
        "query_generator_model": "gemini-2.0-flash",
        "reasoning_model": "gemini-2.0-pro",
    }
)

# 测试 generate_query 函数
try:
    print("\n开始测试 generate_query 函数...")
    result = generate_query(test_state, test_config)
    print("\n测试成功!")
    print(f"生成的查询: {result.get('search_query', [])}")
except Exception as e:
    print(f"\n测试失败: {e}")
    import traceback
    traceback.print_exc()



In [ ]:
"""
Test script to directly test the Gemini API without importing from the project modules.
This will help us verify if the issue is with our code changes or with the Gemini API itself.
"""

import os
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# Check if API key is set
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    print("GEMINI_API_KEY is not set")
    exit(1)
else:
    print("GEMINI_API_KEY is set")

# Import the Gemini API client
try:
    from google.genai import Client
    print("Successfully imported google.genai")
except Exception as e:
    print(f"Error importing google.genai: {e}")
    exit(1)

# Initialize the Gemini API client
genai_client = Client(api_key=gemini_api_key)

# Test a simple query with system instruction for structured output
try:
    print("\nTesting Gemini API with system instruction for structured output...")
    
    # Define the system instruction for structured output
    system_instruction = (
        "You are a search query generator. Generate search queries based on the user's research topic. "
        "Return your response as a JSON object with the following structure: "
        "{ \"query\": [\"query1\", \"query2\", ...] } "
        "where each query is a string that would be effective for web search."
    )
    
    # Define the user prompt
    user_prompt = "Generate search queries for: Who won the UEFA Euro 2024 football tournament?"
    
    # Set the generation config
    generation_config = {
        "temperature": 1.0,
    }
    
    # Call the Gemini API
    response = genai_client.models.generate_content(
        model="gemini-2.0-flash",
        contents=[
            {"role": "system", "parts": [system_instruction]},
            {"role": "user", "parts": [user_prompt]}
        ],
        generation_config=generation_config,
    )
    
    # Print the response
    print(f"Response text: {response.text}")
    
    # Try to parse the JSON response
    try:
        # Find the JSON part in the response
        response_text = response.text
        json_start = response_text.find('{')
        json_end = response_text.rfind('}')
        if json_start != -1 and json_end != -1:
            json_str = response_text[json_start:json_end+1]
            data = json.loads(json_str)
            print(f"Parsed JSON: {json.dumps(data, indent=2)}")
        else:
            print("No JSON found in response")
    except Exception as e:
        print(f"Error parsing JSON: {e}")
    
    print("Test completed successfully!")
except Exception as e:
    print(f"Error testing Gemini API: {e}")
    import traceback
    traceback.print_exc()

In [ ]:

"""
测试不同的 Gemini API 调用格式
"""

import os
import json
from dotenv import load_dotenv
from google import genai

# 加载环境变量
load_dotenv()

# 检查API密钥是否设置
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    print("GEMINI_API_KEY 未设置")
    exit(1)
else:
    print("GEMINI_API_KEY 已设置")

# 初始化 Gemini 客户端
#genai.configure(api_key=gemini_api_key)

# 创建客户端
client = genai.Client()
# 测试不同的格式
def test_format1():
    """测试格式1: 使用 Content 对象"""
    try:
        print("\n测试格式1: 使用 Content 对象")
        
        # 创建 Content 对象
        content = genai.Content(
            parts=[
                genai.Part(text="谁赢得了2024年欧洲杯足球赛?")
            ],
            role="user"
        )
        
        # 调用 API
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=content
        )
        
        print("成功! 响应:", response.text[:100] + "...")
        return True
    except Exception as e:
        print(f"失败: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_format2():
    """测试格式2: 使用字符串"""
    try:
        print("\n测试格式2: 使用字符串")
        
        # 调用 API
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents="谁赢得了2024年欧洲杯足球赛?"
        )
        
        print("成功! 响应:", response.text[:100] + "...")
        return True
    except Exception as e:
        print(f"失败: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_format3():
    """测试格式3: 使用字典列表"""
    try:
        print("\n测试格式3: 使用字典列表")
        
        # 系统指令
        system_instruction = (
            "You are a search query generator. Generate search queries based on the user's research topic. "
            "Return your response as a JSON object with the following structure: "
            "{ \"query\": [\"query1\", \"query2\", ...] } "
            "where each query is a string that would be effective for web search."
        )
        
        # 用户提示
        user_prompt = "谁赢得了2024年欧洲杯足球赛?"
        
        # 调用 API
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[
                {"role": "system", "parts": [{"text": system_instruction}]},
                {"role": "user", "parts": [{"text": user_prompt}]}
            ]
        )
        
        print("成功! 响应:", response.text[:100] + "...")
        return True
    except Exception as e:
        print(f"失败: {e}")
        import traceback
        traceback.print_exc()
        return False

def test_format4():
    """测试格式4: 使用 Part 对象列表"""
    try:
        print("\n测试格式4: 使用 Part 对象列表")
        
        # 系统指令
        system_instruction = (
            "You are a search query generator. Generate search queries based on the user's research topic. "
            "Return your response as a JSON object with the following structure: "
            "{ \"query\": [\"query1\", \"query2\", ...] } "
            "where each query is a string that would be effective for web search."
        )
        
        # 用户提示
        user_prompt = "谁赢得了2024年欧洲杯足球赛?"
        
        # 创建 Content 对象
        system_content = genai.Content(
            parts=[genai.Part(text=system_instruction)],
            role="system"
        )
        
        user_content = genai.Content(
            parts=[genai.Part(text=user_prompt)],
            role="user"
        )
        
        # 调用 API
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=[system_content, user_content]
        )
        
        print("成功! 响应:", response.text[:100] + "...")
        return True
    except Exception as e:
        print(f"失败: {e}")
        import traceback
        traceback.print_exc()
        return False


In [ ]:


print("开始测试不同的 Gemini API 调用格式...")

results = []
results.append(("格式1", test_format1()))
results.append(("格式2", test_format2()))
results.append(("格式3", test_format3()))
results.append(("格式4", test_format4()))

print("\n测试结果汇总:")
for format_name, success in results:
    status = "成功" if success else "失败"
    print(f"{format_name}: {status}")

In [ ]:
"""
测试脚本，用于单独测试 generate_query 函数
"""

import os
import sys
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage

# 导入要测试的函数
from src.agent.graph import generate_query
from src.agent.configuration import Configuration

# 加载环境变量
load_dotenv()

# 检查API密钥是否设置
gemini_api_key = os.getenv("GEMINI_API_KEY")
if gemini_api_key is None:
    print("GEMINI_API_KEY 未设置")
    exit(1)
else:
    print("GEMINI_API_KEY 已设置")

# 创建测试状态
test_state = {
    "messages": [
        HumanMessage(content="谁赢得了2024年欧洲杯足球赛?")
    ]
}

# 创建一个模拟的 Configuration 类
class MockConfiguration:
    def __init__(self):
        self.query_generator_model = "gemini-2.0-flash"
        self.reasoning_model = "gemini-2.0-pro"
        self.number_of_initial_queries = 3
    
    @classmethod
    def from_runnable_config(cls, config):
        return cls()

# 创建一个模拟的 RunnableConfig 类
class MockRunnableConfig:
    def __init__(self):
        self.configurable = MockConfiguration()

# 测试 generate_query 函数
try:
    print("\n开始测试 generate_query 函数...")
    print("\n使用的模型: gemini-2.0-flash")
    print("\n测试状态: ", test_state)
    
    # 添加调试信息
    print("\n开始调用 generate_query...")
    
    # 创建模拟的配置对象
    test_config = MockRunnableConfig()
    
    # 调用函数
    result = generate_query(test_state, test_config)
    print("\n测试成功!")
    print(f"生成的查询: {result.get('query_list', [])}")
except Exception as e:
    print(f"\n测试失败: {e}")
    import traceback
    traceback.print_exc()





In [ ]:
from langchain_openai import AzureChatOpenAI
import os, dotenv

# 加载 .env（若在 backend 目录，可写 dotenv.load_dotenv("backend/.env")）
dotenv.load_dotenv()
# 创建 LLM 实例
llm = AzureChatOpenAI(
    deployment_name='gpt-4.1',
    model_name='gpt-4.1',
    temperature=0.2,
    streaming=False,
)
from langchain_core.messages import HumanMessage
response = llm.invoke([HumanMessage(content="hello")])
print(response)


In [23]:
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph
from typing import TypedDict, Annotated, Literal
from langgraph.graph import END
import operator
import json

# 定义状态
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    next_step: str

# 初始化LLM
#llm = ChatOpenAI(model="gpt-4")

# 路由节点
router_prompt = ChatPromptTemplate.from_messages([
    ("system", "确定用户的意图并路由到相应的专家。如果用户在请求信息，路由到'information'。如果用户正在请求创建内容，路由到'creation'."),
    ("placeholder", "{messages}")
])

def router(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1].content
    
    # 简单判断，不依赖复杂的函数调用
    # 这只是一个简单示例，实际中可能需要更复杂的逻辑
    keywords_for_creation = ["写", "创造", "故事", "创作", "生成"]
    
    # 检查是否包含创作相关关键词
    is_creation = any(keyword in last_message for keyword in keywords_for_creation)
    
    # 确定路由
    destination = "creation" if is_creation else "information"
    
    print(f"路由决策: {destination}, 基于消息: {last_message[:30]}...")
    return {"next_step": destination}

# 信息专家
def information_expert(state: AgentState):
    messages = state["messages"]
    
    information_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个信息专家，能够提供准确、有用的信息。"),
        ("placeholder", "{messages}")
    ])
    
    # 直接在invoke中提供所有参数
    response = information_prompt.pipe(llm).invoke({"messages": messages})
    return {"messages": [AIMessage(content=response.content)]}

def creation_expert(state: AgentState):
    messages = state["messages"]
    
    creation_prompt = ChatPromptTemplate.from_messages([
        ("system", "你是一个创意专家，能够生成原创内容。"),
        ("placeholder", "{messages}")
    ])
    
    # 直接在invoke中提供所有参数
    response = creation_prompt.pipe(llm).invoke({"messages": messages})
    return {"messages": [AIMessage(content=response.content)]}
def should_continue(state):
    # 返回字典，而不是直接返回字符串
    if len(state["messages"]) >= 3:  # 假设最多3轮对话
        return {"next_step": "end"}
    else:
        return {"next_step": "router"}

# 创建图形
graph = StateGraph(AgentState)

# 添加节点
graph.add_node("router", router)
graph.add_node("information", information_expert)
graph.add_node("creation", creation_expert)
graph.add_node("should_continue", should_continue)  # 添加判断节点
# 添加从router到专家节点的条件边

# 添加边和条件路由
graph.set_entry_point("router")
graph.add_conditional_edges(
    "router",
    lambda state: state["next_step"],
    {
        "information": "information",
        "creation": "creation"
    }
)
graph.add_conditional_edges(
    "should_continue",
    lambda state: state["next_step"],  # 从返回的字典中获取next_step字段
    {
        "router": "router",
        "end": END
    }
)
graph.add_edge("information", "should_continue")  # 使用节点名称而不是函数
graph.add_edge("creation", "should_continue")  # 使用节点名称而不是函数


# 编译图形
app = graph.compile()


In [24]:
import dotenv
dotenv.load_dotenv()

# 定义正确的测试数据
# 定义正确的测试数据
state = {
    "messages": [HumanMessage(content="为我写一个短故事")],
    "next_step": ""  # 或者您可以直接设置为 "router"
}

# 打印初始状态，便于调试
print("初始状态:", state)
result = app.invoke(state)
print("结果:", result)


初始状态: {'messages': [HumanMessage(content='为我写一个短故事', additional_kwargs={}, response_metadata={})], 'next_step': ''}
路由决策: creation, 基于消息: 为我写一个短故事...
路由决策: information, 基于消息: 在一个遥远的小镇上，住着一个爱做梦的男孩小宇。他每天放学后，...
结果: {'messages': [HumanMessage(content='为我写一个短故事', additional_kwargs={}, response_metadata={}), AIMessage(content='在一个遥远的小镇上，住着一个爱做梦的男孩小宇。他每天放学后，都会跑到镇外的老橡树下，仰望天空，幻想自己能飞上云端。\n\n有一天，小宇在树下发现了一只受伤的小鸟。他小心翼翼地把小鸟带回家，细心照料。几天后，小鸟渐渐恢复了健康。小宇舍不得让它离开，但他知道，鸟儿属于天空。\n\n终于，小宇鼓起勇气，把小鸟带到橡树下，轻声说：“去吧，飞向你的世界。”小鸟在他手心停留片刻，仿佛在道谢，然后振翅高飞。\n\n小宇望着小鸟消失在云端，心里充满了温暖。他明白了一个道理：有时候，放手也是一种美丽的成全。从那天起，小宇的梦想不再只是飞翔，而是学会了珍惜和成长。', additional_kwargs={}, response_metadata={}), AIMessage(content='在一个安静的小村庄里，住着一位叫阿明的老人。他每天清晨都会在村头的小河边散步，和河里的鸭子们打招呼。\n\n有一天，阿明发现河边多了一只小鸭子，它孤零零地站在石头上，显得很害怕。阿明轻声安慰它，把它带回了家。几天后，小鸭子变得活泼起来，还学会了跟着阿明散步。\n\n春天过去，夏天来临。一天早晨，小鸭子的妈妈带着一群鸭子游到了河边。小鸭子看到妈妈，高兴地扑进了水里。阿明微笑着目送它们远去，心里既有不舍，也有满足。\n\n从那以后，阿明每天散步时，总能看到那只小鸭子在河里向他挥动翅膀。他明白，善良和陪伴，会在心里留下温暖的回忆。', additional_kwargs={}, response_metadata={})], 'next_step': 'end'}


In [ ]:

# 运行图形
try:
    result = app.invoke(state)
    print("结果:", result)
    
    # 如果测试成功，尝试第二轮对话
    if "messages" in result:
        print("AI回复:", result["messages"][-1].content)
        
        # 添加用户回复
        result["messages"].append(HumanMessage(content="让故事更有趣一些"))
        
        # 运行第二轮
        result2 = app.invoke(result)
        print("\n第二轮AI回复:", result2["messages"][-1].content)
except Exception as e:
    print(f"错误类型: {type(e)}")
    print(f"错误信息: {str(e)}")
    import traceback
    traceback.print_exc()


In [ ]:
for i in range(100):
    if i**4 >99999 and i**4 < 1000000:
        # 得到的6尾数 abcdef 计算ab+cd+ef 以及ba+dc+fe
        num = i**4
        num_str = str(num)
        a, b = int(num_str[-6]), int(num_str[-5])
        c, d = int(num_str[-4]), int(num_str[-3])
        e, f = int(num_str[-2]), int(num_str[-1])
        sum1 = 10*a + b + 10*c + d + 10*e + f
        sum2 = 10*b + a + 10*d + c + 10*f + e
        if sum1 == sum2 and int(sum1**0.5)**2 == sum1:
            print(f"i={i}, {num}, sum1={sum1}, sum2={sum2}")